In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [26]:
df = pd.read_csv('./ost_list_endgame.csv', encoding='utf-8')
# df.info()
# df.describe()

---

##### 데이터 전체 분석

In [35]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, f1_score

# df에서 X(data), y(label) 분리

X = df.iloc[:, 27:38]       # 음악의 특성(아티스트, 장르 등 비정형데이터는 제외)
y = df.iloc[:, 2:22]        # 영화의 장르

# train/test 분리

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# 모델 훈련

dt_clf = DecisionTreeClassifier(random_state=42, max_depth=10)
dt_clf.fit(X_train, y_train)

# f1_score 계산
y_pred = dt_clf.predict(X_test)
f1_micro = f1_score(y_test, y_pred, average='micro')
f1_macro = f1_score(y_test, y_pred, average='macro')

# roc_auc_score 계산
y_probs_list = dt_clf.predict_proba(X_test)
y_pred_proba = np.transpose([prob[:, 1] for prob in y_probs_list])
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f'훈련 점수: {dt_clf.score(X_train, y_train):.4f}, 테스트 점수: {dt_clf.score(X_test, y_test):.4f}, f1_MICRO: {f1_micro:.4f},  f1_MACRO: {f1_macro:.4f}, ROC_AUC_SCORE: {roc_auc:.4f}')

# max_depth=10일 때, (0.11955174550712663, 0.08458559256390395)
# 테스트 결과가 매우 낮은데, Subset Accuracy(전체 일치율)을 본다 >>> dt_clf.score()는 멀티 레이블 데이터에서 때 1점으로 계산
# [Drama, Romance]인데 모델이 [Drama]만 맞추고 Romance를 놓치면, 0점 처리 >>> 따라서 점수가 낮을 수 밖에 없을 것...


훈련 점수: 0.1265, 테스트 점수: 0.0878, f1_MICRO: 0.2945,  f1_MACRO: 0.0743, ROC_AUC_SCORE: 0.5587


### Top 3 장르

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, f1_score

X = df.iloc[:, 27:38]
y = df.iloc[:, [2,5,8]]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

dt_clf = DecisionTreeClassifier(random_state=42, max_depth=7)
dt_clf.fit(X_train, y_train)

# f1_score 계산
y_pred = dt_clf.predict(X_test)
f1 = f1_score(y_test, y_pred)
f1_micro = f1_score(y_test, y_pred, average='micro')
f1_macro = f1_score(y_test, y_pred, average='macro')

# roc_auc_score 계산
y_probs_list = dt_clf.predict_proba(X_test)
y_pred_proba = np.transpose([prob[:, 1] for prob in y_probs_list])
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f'훈련 점수: {dt_clf.score(X_train, y_train):.4f}, 테스트 점수: {dt_clf.score(X_test, y_test):.4f}, f1_MICRO: {f1_micro:.4f},  f1_MACRO: {f1_macro:.4f}, ROC_AUC_SCORE: {roc_auc:.4f}')

훈련 점수: 0.3312, 테스트 점수: 0.2940, f1_MICRO: 0.4511,  f1_MACRO: 0.3183, ROC_AUC_SCORE: 0.5662


---

##### 개별 장르에 대한 결정나무 실행

In [40]:
# 개별 장르에 대한 결정나무 실행 >>> SMOTE 실행 전

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, f1_score

models = {}                  # 모델들을 딕셔너리로 저장하기 위해

for i in range(2,22):
    X = df.iloc[:, 27:38]    # 음악의 특성(아티스트, 장르 등 비정형데이터는 제외)
    y = df.iloc[:, i]        # 영화의 장르 하나 선택

    # Train/Test 데이터 분류
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

    # 모델 훈련
    model_name = f'dt_clf_{i}'
    models[model_name] = DecisionTreeClassifier(random_state=42, max_depth=7)

    models[model_name].fit(X_train, y_train)

    # f1_score 계산
    y_pred = models[model_name].predict(X_test)
    f1 = f1_score(y_test, y_pred)

    # roc_auc_score 계산
    y_pred_proba = models[model_name].predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, y_pred_proba)

    print(f'{df.columns[i]} | 훈련 점수: {models[model_name].score(X_train, y_train):.4f}, 테스트 점수: {models[model_name].score(X_test, y_test):.4f}, f1_SCORE: {f1:.4f}, ROC_AUC_SCORE: {roc_auc:.4f}')

print(models)

# F1을 제외한 나머지 평가지표는 모델이 똑똑해서가 아니라, 그냥 무조건 '아니오'라고만 말해도 얻을 수 있는 점수라서 그런 거였다.. F1이 증명해내다.

Action | 훈련 점수: 0.8685, 테스트 점수: 0.8635, f1_SCORE: 0.0265, ROC_AUC_SCORE: 0.5673
Adventure | 훈련 점수: 0.9279, 테스트 점수: 0.9312, f1_SCORE: 0.0263, ROC_AUC_SCORE: 0.5247
Animation | 훈련 점수: 0.9651, 테스트 점수: 0.9653, f1_SCORE: 0.0345, ROC_AUC_SCORE: 0.5213
Comedy | 훈련 점수: 0.6015, 테스트 점수: 0.5393, f1_SCORE: 0.4210, ROC_AUC_SCORE: 0.5500
Crime | 훈련 점수: 0.8893, 테스트 점수: 0.8847, f1_SCORE: 0.0236, ROC_AUC_SCORE: 0.5491
Documentary | 훈련 점수: 0.9243, 테스트 점수: 0.9201, f1_SCORE: 0.0301, ROC_AUC_SCORE: 0.5718
Drama | 훈련 점수: 0.5967, 테스트 점수: 0.5514, f1_SCORE: 0.5527, ROC_AUC_SCORE: 0.5587
Family | 훈련 점수: 0.9253, 테스트 점수: 0.9241, f1_SCORE: 0.0278, ROC_AUC_SCORE: 0.5584
Fantasy | 훈련 점수: 0.9511, 테스트 점수: 0.9548, f1_SCORE: 0.0068, ROC_AUC_SCORE: 0.5318
Foreign | 훈련 점수: 0.9807, 테스트 점수: 0.9782, f1_SCORE: 0.0408, ROC_AUC_SCORE: 0.5739
History | 훈련 점수: 0.9777, 테스트 점수: 0.9782, f1_SCORE: 0.0276, ROC_AUC_SCORE: 0.6513
Horror | 훈련 점수: 0.9428, 테스트 점수: 0.9377, f1_SCORE: 0.0099, ROC_AUC_SCORE: 0.5843
Music | 훈련 점수: 0.8925, 테스트 점

In [39]:
# 개별 장르에 대한 결정나무 실행 >>> SMOTE 실행 후

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, f1_score
from imblearn.over_sampling import SMOTE

models = {}                  # 모델들을 딕셔너리로 저장하기 위해

for i in range(2,22):
    X = df.iloc[:, 27:38]    # 음악의 특성(아티스트, 장르 등 비정형데이터는 제외)
    y = df.iloc[:, i]        # 영화의 장르 하나 선택

    # Train/Test 데이터 분류
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
    
    # SMOTE로 오버샘플링
    smote = SMOTE(sampling_strategy='auto', random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X, y)

    # 모델 훈련
    model_name = f'dt_clf_{i}'
    models[model_name] = DecisionTreeClassifier(random_state=42, max_depth=7)

    models[model_name].fit(X_resampled, y_resampled)

    # f1_score 계산
    y_pred = models[model_name].predict(X_test)
    f1 = f1_score(y_test, y_pred)

    # roc_auc_score 계산
    y_pred_proba = models[model_name].predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, y_pred_proba)

    print(f'{df.columns[i]} | 훈련 점수: {models[model_name].score(X_train, y_train):.4f}, 테스트 점수: {models[model_name].score(X_test, y_test):.4f}, f1_SCORE: {f1:.4f}, ROC_AUC_SCORE: {roc_auc:.4f}')

print(models)

Action | 훈련 점수: 0.6198, 테스트 점수: 0.6263, f1_SCORE: 0.2646, ROC_AUC_SCORE: 0.6026
Adventure | 훈련 점수: 0.5706, 테스트 점수: 0.5752, f1_SCORE: 0.1453, ROC_AUC_SCORE: 0.5861
Animation | 훈련 점수: 0.4945, 테스트 점수: 0.5109, f1_SCORE: 0.0873, ROC_AUC_SCORE: 0.6342
Comedy | 훈련 점수: 0.5796, 테스트 점수: 0.5785, f1_SCORE: 0.5703, ROC_AUC_SCORE: 0.6132
Crime | 훈련 점수: 0.5952, 테스트 점수: 0.6053, f1_SCORE: 0.2246, ROC_AUC_SCORE: 0.5870
Documentary | 훈련 점수: 0.6775, 테스트 점수: 0.6829, f1_SCORE: 0.1716, ROC_AUC_SCORE: 0.6089
Drama | 훈련 점수: 0.5836, 테스트 점수: 0.5805, f1_SCORE: 0.5820, ROC_AUC_SCORE: 0.6155
Family | 훈련 점수: 0.5263, 테스트 점수: 0.5325, f1_SCORE: 0.1475, ROC_AUC_SCORE: 0.5712
Fantasy | 훈련 점수: 0.5544, 테스트 점수: 0.5665, f1_SCORE: 0.1055, ROC_AUC_SCORE: 0.6102
Foreign | 훈련 점수: 0.6777, 테스트 점수: 0.6844, f1_SCORE: 0.0600, ROC_AUC_SCORE: 0.6558
History | 훈련 점수: 0.6145, 테스트 점수: 0.6211, f1_SCORE: 0.0700, ROC_AUC_SCORE: 0.7115
Horror | 훈련 점수: 0.6206, 테스트 점수: 0.6204, f1_SCORE: 0.1528, ROC_AUC_SCORE: 0.6578
Music | 훈련 점수: 0.6033, 테스트 점

---